In [ ]:
!pip install votekit

# Running an Election in VoteKit

As it turns out, once you know how to generate a `PreferenceProfile` object running an election
is pretty straightforward. Whenever you call the constructor for an `Election` object
the entire election will actually be run, and you will be able to access the results
instantly.

In [ ]:
from gerrychain import Graph
import jsonlines as jl
import votekit.elections as elec
from votekit import PreferenceProfile
import votekit.ballot_generator as bg

In [ ]:
bloc_voter_prop = {
    "W": 0.2,
    "C": 0.8
}
cohesion_parameters = {
    "W": {
        "W": 0.7,
        "C": 0.3
    },
    "C": {
        "W": 0.7,
        "C": 0.3
    }
}
alphas = {
    "W": {
        "W": 1,
        "C": 1
    },
    "C": {
        "W": 1,
        "C": 1
    }
}
slate_to_candidates = {
    "W": [
        "W1",
        "W2",
        "W3"
    ],
    "C": [
        "C1",
        "C2"
    ]
}

In [ ]:
profile = bg.slate_PlackettLuce.from_params(
    bloc_voter_prop=bloc_voter_prop,
    cohesion_parameters=cohesion_parameters,
    alphas=alphas,
    slate_to_candidates=slate_to_candidates
).generate_profile(
    number_of_ballots=1000
)

In [ ]:
election = elec.STV(profile, m=3)

In any multi-round election, you can then get all of the information about
how the election progressed by accessing the `election_state` attribute of the
`Election` object.

In [ ]:
election

In [ ]:
election.get_elected()

In [ ]:
for i in range(4):
    print()
    print(election.election_states[i])
    print(election.election_states[i].elected)
    print(election.election_states[i].remaining)
    print(election.election_states[i].scores)

So, all we now need to do is figure out a good way of generating a lot of samples
from a lot of different settings. Gathering a lot of samples is easy: just
run the ballot generator and the election a bunch of times and then save the results.

In [ ]:
from tqdm.notebook import tqdm
with jl.open('election_results.jsonl', 'w') as writer:
    # for _ in range(10):
    for _ in tqdm(range(30)):
        profile = bg.slate_PlackettLuce.from_params(
            bloc_voter_prop=bloc_voter_prop,
            cohesion_parameters=cohesion_parameters,
            alphas=alphas,
            slate_to_candidates=slate_to_candidates
        ).generate_profile(
            number_of_ballots=10000
        )
        election = elec.STV(profile, m=3)
        
        writer.write({
            "winners": [winner for winner_set in election.get_elected() for winner in winner_set],
        })

So the question then becomes, how do we make better predictions?
Well, the first thing that we need to do is gather some information about
the location in question. We'll look at our dual graph file in this
notebook, but commonly, you'll need something like census data to agument
your work.

In [ ]:
graph = Graph.from_json("../../../../data/gerrymandria.json")

In [ ]:
graph.nodes[0]

A good starting point for us here is to just get an estimat on the
state-wide POCVAP and WVAP values.

In [ ]:
wvap_total = sum(d["WVAP"] for _, d in graph.nodes(data=True))
pocvap_total = sum(d["POCVAP"] for _, d in graph.nodes(data=True))
total_pop = sum(d["TOTPOP"] for _, d in graph.nodes(data=True))
print(f"Total WVAP: {wvap_total}")
print(f"\tTotal WCAP %: {wvap_total / total_pop * 100:.2f}")
print(f"Total POCVAP: {pocvap_total}")
print(f"\tTotal POCAP %: {pocvap_total / total_pop * 100:.2f}")

These would be good starting points for some of the parameters for our
ballot generator, namely the `bloc_voter_prop` parameter.

In [ ]:
bloc_voter_prop = {
    "W": 0.75,
    "C": 0.25
}
cohesion_parameters = {
    "W": {
        "W": 0.7,
        "C": 0.3
    },
    "C": {
        "W": 0.7,
        "C": 0.3
    }
}
alphas = {
    "W": {
        "W": 1,
        "C": 1
    },
    "C": {
        "W": 1,
        "C": 1
    }
}
slate_to_candidates = {
    "W": [
        "W1",
        "W2",
        "W3"
    ],
    "C": [
        "C1",
        "C2"
    ]
}

In [ ]:
ballot_generator_kwargs = dict(
    bloc_voter_prop=bloc_voter_prop,
    cohesion_parameters=cohesion_parameters,
    alphas=alphas,
    slate_to_candidates=slate_to_candidates
)

In [ ]:
with jl.open('election_results2.jsonl', 'w') as writer:
    # for _ in range(10):
    for _ in tqdm(range(30)):
        profile = bg.slate_PlackettLuce.from_params(
            **ballot_generator_kwargs
        ).generate_profile(
            number_of_ballots=10000
        )
        election = elec.STV(profile, m=3)
        
        writer.write({
            "winners": [winner for winner_set in election.get_elected() for winner in winner_set],
        })

Okay, this is a great starting point, but putting together a bunch of `for`
loops in a single notebook is difficult to read, audit, and scale.
So we are going to need a better way to do and organize this if we want
to be able to keep track of all of the information we are generating.